In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

In [3]:
df = pd.read_csv('../data/clean_data.csv')
df.head()

,id,date,phones_num,phb_fraud,phb_dirty,phb_all_bad,phb_all,phb_bad_perc,found_phones,phb_fraud_dir,...,defendant_case_perc,rating_ya,ratings_cnt_ya,reviews_cnt_ya,rating_yell,reviews_cnt_yell,rating_2gis,reviews_cnt_2gis,rating_avito,reviews_cnt_avito
0,a782615660478a,2022-10-30 00:00:00,1,0.0,2.0,2.0,1498.0,0.13,1.0,0.0,...,0.0,-1.00,0.0,15.0,4.000000,0.0,0.000000,0.0,-1.0,-1.0
1,a7706752171a,2022-01-15 00:00:00,16,0.0,0.0,0.0,2421.0,0.00,16.0,0.0,...,0.0,4.15,114.0,39.0,3.700000,1.0,0.142857,2.0,4.8,5.0
2,a745212602258a,2021-06-03 00:00:00,4,0.0,0.0,0.0,1503.0,0.00,3.0,0.0,...,0.0,-1.00,0.0,4.0,4.000000,0.0,0.000000,0.0,-1.0,-1.0
3,a2466264323a,2022-09-25 00:00:00,3,0.0,0.0,0.0,77.0,0.00,2.0,0.0,...,0.0,-1.00,0.0,2.0,4.500000,0.0,4.800000,5.0,-1.0,-1.0
4,a7728551510a,2022-04-28 00:00:00,161,4.0,2.0,6.0,17842.0,0.03,145.0,0.0,...,0.0,4.15,21338.0,5772.0,4.244828,308.0,1.358362,2943.0,-1.0,-1.0


In [4]:
rating_columns = [
    "rating_ya",
    "rating_yell",
    "rating_avito",
    "rating_2gis",
]

reviews_count_columns = [
    "reviews_cnt_ya",
    "reviews_cnt_yell",
    "reviews_cnt_2gis",
    "reviews_cnt_avito",
    "ratings_cnt_ya"
]

phones = [
    "phb_bad_perc_dir",
    "phb_all_dir",  # !
    "phb_all_bad_dir",  # !
    "phb_dirty_dir",
    "phb_fraud_dir",
    "phb_bad_perc",
    "found_phones_dir",
    "phb_all",
    "phb_all_bad",
    "phb_dirty",
    "phb_fraud",
    "found_phones",
    "phones_num"

]

bank_and_police = [
    "total_defendant_case_amt",
    "defendant_case_perc",
    "total_defendant_case_cnt",
    "total_case_cnt",
    "mvd_request_num",
    "complaints_num_dir",
    "complaints_num",
    "claims_percent",
    "appl_num",
    "bad_rep_tcb"
]

data_without_description = [
    "claims_num"
]

In [5]:
df['all_reviews_cnt'] = df[reviews_count_columns].sum(axis=1)

In [6]:
def get_mean_rating(row):
    sm = n = 0
    for r in row[rating_columns]:
        if r > 0:
            sm += r
            n += 1
    if n == 0:
        return 0
    return sm / n

df['mean_rating'] = df.apply(lambda x: get_mean_rating(x), axis=1)

In [7]:
size_columns = ['phb_all',  'appl_num', 'all_reviews_cnt']
has_reviews = df[df['all_reviews_cnt'] > 0]

q90 = has_reviews[size_columns].quantile(0.9)
q75 = has_reviews[size_columns].quantile(0.75)
q50 = has_reviews[size_columns].quantile(0.50)
q25 = has_reviews[size_columns].quantile(0.25)

q = has_reviews[size_columns].quantile([0.25, 0.5, 0.75, 0.9])
q.at[0.75, 'phb_all']

3697.0

In [8]:
real_huge_companies = df[(df['phb_all'] > q90['phb_all']) & (df['appl_num'] > q90['appl_num']) & (df['all_reviews_cnt'] > q90['all_reviews_cnt'])]
real_huge_companies.describe()

,phones_num,phb_fraud,phb_dirty,phb_all_bad,phb_all,phb_bad_perc,found_phones,phb_fraud_dir,phb_dirty_dir,phb_all_bad_dir,...,ratings_cnt_ya,reviews_cnt_ya,rating_yell,reviews_cnt_yell,rating_2gis,reviews_cnt_2gis,rating_avito,reviews_cnt_avito,all_reviews_cnt,mean_rating
count,11.00000,11.000000,11.00000,11.000000,1.100000e+01,11.00000,11.000000,11.000000,11.000000,11.000000,...,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000
mean,1035.00000,454.454545,350.00000,804.454545,6.945040e+05,0.06000,993.909091,0.727273,3.272727,4.000000,...,10021.545455,3106.000000,4.223635,26.818182,1.425632,759.272727,-0.544444,18.545455,13932.181818,3.252823
std,2367.55963,1287.842565,848.20316,2133.215946,1.678388e+06,0.04899,2301.069728,2.412091,10.854408,13.266499,...,16752.205672,5182.214025,0.104146,39.104522,1.507144,1082.265410,1.510907,64.824939,22689.927205,0.583005
min,6.00000,0.000000,0.00000,0.000000,1.092300e+04,0.00000,6.000000,0.000000,0.000000,0.000000,...,402.000000,140.000000,4.000000,0.000000,0.071642,1.000000,-1.000000,-1.000000,1046.000000,2.694167
25%,55.00000,1.000000,3.00000,4.000000,2.990600e+04,0.01500,53.000000,0.000000,0.000000,0.000000,...,973.000000,327.500000,4.230598,1.000000,0.570321,33.500000,-1.000000,-1.000000,1272.000000,2.844690
50%,368.00000,26.000000,22.00000,62.000000,7.838400e+04,0.08000,301.000000,0.000000,0.000000,0.000000,...,2385.000000,741.000000,4.240574,9.000000,0.759524,120.000000,-1.000000,-1.000000,3247.000000,3.059784
75%,572.00000,125.500000,239.50000,365.000000,3.175160e+05,0.09500,540.500000,0.000000,0.000000,0.000000,...,13268.000000,4359.500000,4.276551,42.000000,1.711729,1169.000000,-1.000000,-1.000000,19010.000000,3.480976
max,8090.00000,4326.000000,2867.00000,7193.000000,5.693618e+06,0.13000,7851.000000,8.000000,36.000000,44.000000,...,56712.000000,17681.000000,4.350000,131.000000,4.700000,2677.000000,4.011111,214.000000,76542.000000,4.400000


In [9]:
huge_companies = df[(df['phb_all'] > q90['phb_all']) | (df['appl_num'] > q90['appl_num']) | (df['all_reviews_cnt'] > q90['all_reviews_cnt'])]
huge_companies.describe()

,phones_num,phb_fraud,phb_dirty,phb_all_bad,phb_all,phb_bad_perc,found_phones,phb_fraud_dir,phb_dirty_dir,phb_all_bad_dir,...,ratings_cnt_ya,reviews_cnt_ya,rating_yell,reviews_cnt_yell,rating_2gis,reviews_cnt_2gis,rating_avito,reviews_cnt_avito,all_reviews_cnt,mean_rating
count,2665.000000,2665.000000,2665.000000,2665.000000,2.665000e+03,2665.000000,2665.000000,2665.000000,2665.000000,2665.000000,...,2.665000e+03,2665.000000,2665.000000,2665.000000,2665.000000,2665.000000,2665.000000,2665.000000,2.665000e+03,2665.000000
mean,51.488931,13.381614,7.113321,20.494934,1.818190e+04,0.096608,42.144465,0.634522,0.435272,1.069794,...,1.850109e+03,574.296435,1.029702,5.992871,-0.150009,43.792120,-0.835121,0.339587,2.474530e+03,1.646742
std,356.431866,120.375590,65.213814,180.041527,1.417487e+05,0.553600,311.425313,6.310295,3.525597,9.208484,...,2.734710e+04,8014.958277,2.575928,43.729842,1.314240,380.944862,0.933234,18.276978,3.564468e+04,1.941008
min,1.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,-1.000000e+00,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-5.000000e+00,0.000000
25%,1.000000,0.000000,0.000000,0.000000,4.840000e+02,0.000000,1.000000,0.000000,0.000000,0.000000,...,-1.000000e+00,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-5.000000e+00,0.000000
50%,2.000000,0.000000,0.000000,0.000000,1.200000e+03,0.000000,2.000000,0.000000,0.000000,0.000000,...,-1.000000e+00,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-4.000000e+00,0.000000
75%,6.000000,0.000000,0.000000,1.000000,3.500000e+03,0.030000,5.000000,0.000000,0.000000,0.000000,...,2.400000e+01,11.000000,4.214667,0.000000,0.000000,0.000000,-1.000000,-1.000000,5.300000e+01,3.800000
max,8090.000000,4326.000000,2867.000000,7193.000000,5.693618e+06,14.490000,7851.000000,212.000000,92.000000,293.000000,...,1.126416e+06,326694.000000,5.000000,964.000000,5.000000,14269.000000,5.000000,707.000000,1.467875e+06,5.000000


In [10]:
susp_reviews = df[(df['phb_all'] < q25['phb_all']) & (df['all_reviews_cnt'] > q90['all_reviews_cnt'])]
susp_reviews[['phb_all', 'all_reviews_cnt', 'bad_rep_tcb']]

,phb_all,all_reviews_cnt,bad_rep_tcb
107,116.0,1264.0,0.0
856,15.0,11866.0,0.0
1238,112.0,2538.0,0.0
1437,520.0,848.0,0.0
1936,380.0,879.0,0.0
1978,22.0,1022.0,0.0
2035,18.0,17618.0,0.0
2313,6.0,1467875.0,0.0
2531,97.0,4477.0,0.0
2596,145.0,939.0,0.0


In [11]:
q = df.phb_all.quantile([0, 0.25, 0.50, 0.75, 1])
print(q)
def get_size(phb_cnt):
    for i, v in enumerate(q.values):
        if phb_cnt <= v:
            return i

df['popularity'] = df['phb_all'].apply(lambda x: get_size(x))
df.head()

0.00          0.0
0.25        228.0
0.50        662.0
0.75       1549.0
1.00    5693618.0
Name: phb_all, dtype: float64


,id,date,phones_num,phb_fraud,phb_dirty,phb_all_bad,phb_all,phb_bad_perc,found_phones,phb_fraud_dir,...,reviews_cnt_ya,rating_yell,reviews_cnt_yell,rating_2gis,reviews_cnt_2gis,rating_avito,reviews_cnt_avito,all_reviews_cnt,mean_rating,popularity
0,a782615660478a,2022-10-30 00:00:00,1,0.0,2.0,2.0,1498.0,0.13,1.0,0.0,...,15.0,4.000000,0.0,0.000000,0.0,-1.0,-1.0,14.0,4.000000,3
1,a7706752171a,2022-01-15 00:00:00,16,0.0,0.0,0.0,2421.0,0.00,16.0,0.0,...,39.0,3.700000,1.0,0.142857,2.0,4.8,5.0,161.0,3.198214,4
2,a745212602258a,2021-06-03 00:00:00,4,0.0,0.0,0.0,1503.0,0.00,3.0,0.0,...,4.0,4.000000,0.0,0.000000,0.0,-1.0,-1.0,3.0,4.000000,3
3,a2466264323a,2022-09-25 00:00:00,3,0.0,0.0,0.0,77.0,0.00,2.0,0.0,...,2.0,4.500000,0.0,4.800000,5.0,-1.0,-1.0,6.0,4.650000,1
4,a7728551510a,2022-04-28 00:00:00,161,4.0,2.0,6.0,17842.0,0.03,145.0,0.0,...,5772.0,4.244828,308.0,1.358362,2943.0,-1.0,-1.0,30360.0,3.251063,4


In [30]:
def score(df, column_name):  # [0; 10]
    mn = df[df][column_name].min()
    mx = df[column_name].max()
    df[column_name + "_score"] = 10 * (1 - (df[column_name] - mn) / (mx - mn))

In [31]:
columns_to_score = ['phb_bad_perc', 'phb_bad_perc_dir', 'claims_percent']

In [32]:
for col in columns_to_score:
    score(df, col)

In [33]:
df.head()

,id,date,phones_num,phb_fraud,phb_dirty,phb_all_bad,phb_all,phb_bad_perc,found_phones,phb_fraud_dir,...,rating_2gis,reviews_cnt_2gis,rating_avito,reviews_cnt_avito,all_reviews_cnt,mean_rating,popularity,phb_bad_perc_score,phb_bad_perc_dir_score,claims_percent_score
0,a782615660478a,2022-10-30 00:00:00,1,0.0,2.0,2.0,1498.0,0.13,1.0,0.0,...,0.000000,0.0,-1.0,-1.0,14.0,4.000000,3,9.974,10.00,10.0
1,a7706752171a,2022-01-15 00:00:00,16,0.0,0.0,0.0,2421.0,0.00,16.0,0.0,...,0.142857,2.0,4.8,5.0,161.0,3.198214,4,10.000,10.00,10.0
2,a745212602258a,2021-06-03 00:00:00,4,0.0,0.0,0.0,1503.0,0.00,3.0,0.0,...,0.000000,0.0,-1.0,-1.0,3.0,4.000000,3,10.000,10.00,10.0
3,a2466264323a,2022-09-25 00:00:00,3,0.0,0.0,0.0,77.0,0.00,2.0,0.0,...,4.800000,5.0,-1.0,-1.0,6.0,4.650000,1,10.000,9.96,10.0
4,a7728551510a,2022-04-28 00:00:00,161,4.0,2.0,6.0,17842.0,0.03,145.0,0.0,...,1.358362,2943.0,-1.0,-1.0,30360.0,3.251063,4,9.994,10.00,10.0


In [34]:
df[df['all_reviews_cnt'] > 0]

,id,date,phones_num,phb_fraud,phb_dirty,phb_all_bad,phb_all,phb_bad_perc,found_phones,phb_fraud_dir,...,rating_2gis,reviews_cnt_2gis,rating_avito,reviews_cnt_avito,all_reviews_cnt,mean_rating,popularity,phb_bad_perc_score,phb_bad_perc_dir_score,claims_percent_score
0,a782615660478a,2022-10-30 00:00:00,1,0.0,2.0,2.0,1498.0,0.13,1.0,0.0,...,0.000000,0.0,-1.0,-1.0,14.0,4.000000,3,9.974,10.00,10.0
1,a7706752171a,2022-01-15 00:00:00,16,0.0,0.0,0.0,2421.0,0.00,16.0,0.0,...,0.142857,2.0,4.8,5.0,161.0,3.198214,4,10.000,10.00,10.0
2,a745212602258a,2021-06-03 00:00:00,4,0.0,0.0,0.0,1503.0,0.00,3.0,0.0,...,0.000000,0.0,-1.0,-1.0,3.0,4.000000,3,10.000,10.00,10.0
3,a2466264323a,2022-09-25 00:00:00,3,0.0,0.0,0.0,77.0,0.00,2.0,0.0,...,4.800000,5.0,-1.0,-1.0,6.0,4.650000,1,10.000,9.96,10.0
4,a7728551510a,2022-04-28 00:00:00,161,4.0,2.0,6.0,17842.0,0.03,145.0,0.0,...,1.358362,2943.0,-1.0,-1.0,30360.0,3.251063,4,9.994,10.00,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14799,a590300406972a,2022-05-27 00:00:00,5,0.0,0.0,0.0,979.0,0.00,5.0,0.0,...,0.000000,0.0,-1.0,-1.0,9.0,4.500000,3,10.000,10.00,10.0
14802,a4825125008a,2022-04-28 00:00:00,34,0.0,0.0,0.0,1590.0,0.00,31.0,0.0,...,0.090909,3.0,-1.0,-1.0,4964.0,2.486970,4,10.000,10.00,10.0
14887,a7743857750a,2022-07-07 00:00:00,9,0.0,0.0,0.0,3354.0,0.00,5.0,0.0,...,0.000000,0.0,-1.0,-1.0,5.0,4.000000,4,10.000,10.00,10.0
14888,a6606003385a,2021-11-16 00:00:00,165,4.0,1.0,5.0,20287.0,0.02,155.0,0.0,...,0.000000,0.0,-1.0,-1.0,730.0,4.351429,4,9.996,10.00,10.0
